In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import FeatureHasher
from nltk.corpus import stopwords as sw
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import RussianStemmer
from jamspell import TSpellCorrector
import pymorphy2
import re
import numpy as np
from sklearn.cluster import KMeans
stopwords = sw.words('russian')
jam = TSpellCorrector()
jam.LoadLangModel('..//Data//jamspell_model.bin')
tokenizer = RegexpTokenizer(r'\w+')
morph = pymorphy2.MorphAnalyzer()
stemmer = RussianStemmer()

In [60]:
custom_stopwords = ['сколько','во-сколько']
def preprocess_word(word):
    return stemmer.stem(morph.parse(jam.FixFragment(word))[0].normal_form)

def read_to_list(filename):
    with open(filename, 'r',encoding='utf8') as f:
        content = f.readlines()
        content = [x.strip() for x in content] 
    return content

def preprocess(list_):
    new_list = []
    for l in list_:
        words = tokenizer.tokenize(l)
        
        new_words = [preprocess_word(word) for word in words 
                        if morph.parse(jam.FixFragment(word))[0].normal_form not in stopwords 
                        and not any(char.isdigit() for char in word)
                        and not bool(re.search(r'[a-zA-Z]', word))
                        and morph.parse(jam.FixFragment(word))[0].normal_form not in custom_stopwords
                    ]
        new_list.append(' '.join(w for w in new_words))
    return new_list

In [61]:
questions = read_to_list('..//Data//abiturient.ru.txt')
questions = preprocess(questions)

In [62]:
vectorizer = TfidfVectorizer(min_df=3,ngram_range=(1,1))
X = vectorizer.fit_transform(questions)
print(len(vectorizer.get_feature_names()))
print(vectorizer.get_feature_names())

567
['абитуриент', 'август', 'адрес', 'аккредитац', 'английск', 'анкет', 'апрел', 'арм', 'аттестат', 'баз', 'бакалавр', 'бал', 'балл', 'безопасн', 'билет', 'благодар', 'ближн', 'больш', 'боя', 'брат', 'будущ', 'бюджет', 'бюджетн', 'важн', 'вакантн', 'ваш', 'ввид', 'ве', 'везт', 'вероятн', 'вест', 'вечер', 'вечерн', 'взят', 'вид', 'видет', 'вмест', 'вниман', 'внутрен', 'воен', 'военкомат', 'возможн', 'возникнут', 'волн', 'вообщ', 'вопрос', 'востребова', 'врем', 'времен', 'вступительн', 'вуз', 'выбра', 'выдава', 'выдел', 'выйт', 'вылож', 'высла', 'высок', 'выстав', 'высыла', 'вычислительн', 'выш', 'го', 'говор', 'год', 'город', 'готов', 'гражданин', 'граф', 'дава', 'дал', 'далек', 'дальн', 'дан', 'дат', 'двер', 'действ', 'действительн', 'дека', 'дел', 'дела', 'ден', 'деньг', 'дизайн', 'дипл', 'дисциплин', 'длит', 'дмитр', 'добр', 'дойт', 'документ', 'долг', 'должн', 'дом', 'дополнительн', 'допуст', 'достаточн', 'дума', 'егэ', 'еха', 'ещ', 'ждат', 'желан', 'жизн', 'жил', 'жит', 'жител', '

In [63]:
kmeans = KMeans(n_clusters=10, random_state=0).fit(X)

In [64]:
dict = {
    0:"Шанс/Балл/Таблица с фамилиями/Расположение в таблице(Порядок, приоритеты)",
    1:"Общежитие/бюджетное место",
    2:"Документы",
    3:"Орг вопросы",
    4:"Списки зачисления",
    5:"ЕГЭ/зая",
    6:"Стоимость",
    7:"",
    8:"Зачисление, приоритеты/Оригинал-копия",
    9:"",
    10:""
}

one = []
two = []
three = []
four = []
five = []
six = []
seven = []
eight = []
nine = []
ten = []

In [65]:
for j,k in zip(range(808),kmeans.labels_):
    if k==0:
        one.append(j)
    elif k==1:
        two.append(j)
    elif k==2:
        three.append(j)
    elif k==3:
        four.append(j)
    elif k==4:
        five.append(j)
    elif k==5:
        six.append(j)
    elif k==6:
        seven.append(j)
    elif k==7:
        eight.append(j)
    elif k==8:
        nine.append(j)
    elif k==8:
        ten.append(j)


In [72]:
qs = read_to_list('..//Data//abiturient.ru.txt')
for i in six:
    print("--",qs[i])

-- Здравствуйте. У меня на сайте в списках, подавших заявлений на поступлении, в таблице указано под графой ТИП- "ВИ", что это значит? Я поступаю по результатам ЕГЭ, моя заявка на поступление пришла сегодня.
-- Я узнал, что в этом году не выдаются сертификаты ЕГЭ. Нужны ли какие-либо данные, кроме паспортных, для заверения результатов  ЕГЭ(т.к. для просмотра результатов на официальном портале используется номер паспорта)?
-- Здравствуйте, как нам поступить в нашей ситуации: ЕГЭ сдаём во второй волне (поступаем после техникума) с 8.07 по 15.07, 19.07 резервный день для не сдавших по уважительной причине, на получение свидетельства ЕГЭ 5-7 дней, а приём документов для поступающих по результатам ЕГЭ до 25.07, получается, что мы не успеваем поступить(мы иногородние. Как нам поступить?
-- Скажите, а свидетельства ЕГЭ МИЭТ уже проверял в  федеральной базе?
-- Здравствуйте, нужно ли вместе с остальными документами посылать сертификат о сдачи егэ?
-- Здравствуйте. Я заполняю заявление,но у мен